### 웹 크롤링 하는 의식의 흐름
    정보를 가져오고자 하는 url 정의
    url 정보로 requests로 정보 요청 => text 정보 얻음
    text 정보를 BeautifulSoup를 이용해 html로 변환
    html에서 우리가 필요한 정보만 선별

### 기본 사용법

In [2]:
import numpy as np
import requests
from bs4 import BeautifulSoup

In [4]:
# url정의
url = 'https://www.naver.com/'

# requsts로 url에 정보요청
response = requests.get(url).text

# 정보를 html 변환 (보기 쉽게)
html = BeautifulSoup(response, 'html.parser')

# html 내에서 우리가 보고 싶은 정보만 선별
html.select('img')

#### 셀렉터
    용도 : html에서 내가 원하는 내용을 찾아내기 위해서
    <span class="news" id="1234">비비고 왕교자</span>

    단일 셀렉터
    
    html.select('span')
    tag : span
    class(별명, 그룹명) : .news
    id(고유값) : #1234

#### 복합 셀렉터
    1. 조합 셀렉터
    <span>1</span>
    <span class="txt">2</span>
    <em class="txt">3</em>
    
    태그 이름이 span이고 클래스 이름은 txt인 라인을 찾고 싶다. : span.txt 
    li 태그 중에서 id가 name 인 라인을 찾고\ 싶다. : li#name

    2. 경로 셀렉터
    <ul>
        <li><span>이걸 찾으려면?</span></li>
    </ul>
    <span>이건 아님</span>

    ul 태그안 li 태그 안 span 라인을 찾는다
    ul > li > span 혹은 ul li span

### 로또 번호 1~10회차 가져오기

In [57]:
import time

In [58]:
total_lotto_list = []
for i in range(1, 10):
    
    # 차단막는 코드
    seed = np.random.randint(100)
    np.random.seed(seed)
    a = np.random.randint(5) # 난수가 생성 생성된 난수에 따라서 요청하는 시간 딜레이
    time.sleep(a)
    
    lotto = []
    url = f'https://search.daum.net/search?w=tot&DA=LOT&rtmaxcoll=LOT&&q={i}회차%20로또'
    response = requests.get(url)
    
    if response.status_code == requests.codes.ok: # 연결 성공했는지 확인
        print('접속성공')
        
        html = BeautifulSoup(response.text, 'html.parser')
        numbers = html.select('span.ball')
        for num in numbers:
            lotto.append(num.text)
        total_lotto_list.append(lotto)
        
    else:
        break

접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공
접속성공


In [50]:
requests.codes.ok
# 100 우리 이런정보 내주는거야
# 200 성공
# 300 우리 이 사이트 이리루 이사했어 일루가
# 400 유저가 요청을 잘못한경우
# 500 서버 문제

200

### 네이버 영화 평점(리뷰) 크롤링

In [5]:
import urllib
url = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=184318&target=after'
soup = BeautifulSoup(urllib.request.urlopen(url), 'html.parser') # requests 사용을 거치지 않는 방법
review_list = soup.select('td.title')

print(review_list)

[<td class="title">
<a class="movie color_b" href="/movie/bi/mi/basic.nhn?code=184318">블랙 위도우</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
</div>
<br/>이시대 최고의히어로영화 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('chlw****', '3I550Sh+qemiZAJKjFbxKBGlqO6osY5u+SUInREoQH0=', '이시대 최고의히어로영화', '17588999', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>, <td class="title">
<a class="movie color_b" href="/movie/bi/mi/basic.nhn?code=184318">블랙 위도우</a>
<div class="list_netizen_score">
<span class="st_off"><span class="st_on" style="width:100%">별점 - 총 10점 중</span></span><em>10</em>
</div>
<br/>기다린 보람이 있네요~ 너무 재미있어요^^ 
			
			
			
				
				
				
				<a class="report" href="#" onclick="report('eipr****', 'nVdl9oUKzM7mnjr8sBPv/81ASzSLqTU9o+G2+zhEEDY=', '기다린 보람이 있네요~ 너무 재미있어요^^', '17588998', 'point_after');" style="color:#8F8F8F" title="새 창">신고</a>
</td>, <td class="titl

In [6]:
fin_review_list = []

for review in review_list:
    text = review.text

    count = 0
    for idx, ch in enumerate(text):
        # 필요없는 문자 제거 과정
        if ch == '\n':
            count += 1
        if count == 5:
            break

    new_text = text[idx+1:]

    # 필요없는 문자 제거 과정
    new_text = new_text.replace(' \n\t\t\t\n\t\t\t\n\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t신고\n', '')
    fin_review_list.append(new_text)
    
print(fin_review_list)

['이시대 최고의히어로영화', '기다린 보람이 있네요~ 너무 재미있어요^^', 'Girls, Make a choice by your own', '이거 극장에서 안 보면 후회한다 영화 대유잼', '존잼입니다꼭보십시오', '엔드게임 때 생각도 나고….그냥너무재밌어요 꼭 보세요 돈 하나도 안 아까워요….', '액션은 그냥 펑범한데 스토리는 나쁘지 않네요 다음편을 위한 징검다리 느낌도 있거요', '스토리가 조금 부족하긴 했는데 액션이나 블랙위도우의 단독영화라 너무 좋았습니다.', "살면서 본 영화 중에서 가장 '영화'다운 영화였다.현대의 기술 발전이 영화를 영화로써의 연출이 가능하게 만들었다고 생각한다. 몰입되는 액션은 물론이고, 개연성, 감정 묘사 같은 상상하는 연출을 구현 가능할 수 있게 만들었다.첩보물처럼 디테일한 연출이 취향이였고, 그 동안 마블 영화를 재밌게 봤던 관람객들에게 정말 재밌는 영화일 것이다", '아 뽕찬다 어벤져스 정주행가야지']


## 동적 페이지 크롤링
### 네이버 데이터랩 인기 검색어 가져오기
참조 : https://luminitworld.tistory.com/88

In [7]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import json

In [74]:
# 개발자 도구 - Network - Headers - General에서 확인

url = 'https://datalab.naver.com/shoppingInsight/getKeywordRank.naver?timeUnit=date&cid=50000001'

header = {
    'referer': 'https://datalab.naver.com/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

In [77]:
response = requests.post(url, headers=header)
if response.status_code == requests.codes.ok:
    print("접속 성공")

접속 성공


In [78]:
response.text

'[{"message":null,"statusCode":200,"returnCode":0,"date":"2021/06/28","datetime":"2021.06.28.(월)","range":"","ranks":[{"rank":1,"keyword":"크록스","linkId":"크록스"},{"rank":2,"keyword":"나이키운동화","linkId":"나이키운동화"},{"rank":3,"keyword":"뉴발란스토앤토","linkId":"뉴발란스토앤토"},{"rank":4,"keyword":"양산","linkId":"양산"},{"rank":5,"keyword":"슬리퍼","linkId":"슬리퍼"},{"rank":6,"keyword":"크록스샌들","linkId":"크록스샌들"},{"rank":7,"keyword":"아쿠아슈즈","linkId":"아쿠아슈즈"},{"rank":8,"keyword":"샌들","linkId":"샌들"},{"rank":9,"keyword":"모자","linkId":"모자"},{"rank":10,"keyword":"우포스쪼리","linkId":"우포스쪼리"}]},{"message":null,"statusCode":200,"returnCode":0,"date":"2021/06/29","datetime":"2021.06.29.(화)","range":"","ranks":[{"rank":1,"keyword":"크록스","linkId":"크록스"},{"rank":2,"keyword":"나이키운동화","linkId":"나이키운동화"},{"rank":3,"keyword":"뉴발란스토앤토","linkId":"뉴발란스토앤토"},{"rank":4,"keyword":"슬리퍼","linkId":"슬리퍼"},{"rank":5,"keyword":"양산","linkId":"양산"},{"rank":6,"keyword":"크록스샌들","linkId":"크록스샌들"},{"rank":7,"keyword":"샌들","linkId":"샌들"},{"rank":8,"keyw

In [79]:
data = json.loads(response.text) # json 형태로 변환
data

[{'message': None,
  'statusCode': 200,
  'returnCode': 0,
  'date': '2021/06/28',
  'datetime': '2021.06.28.(월)',
  'range': '',
  'ranks': [{'rank': 1, 'keyword': '크록스', 'linkId': '크록스'},
   {'rank': 2, 'keyword': '나이키운동화', 'linkId': '나이키운동화'},
   {'rank': 3, 'keyword': '뉴발란스토앤토', 'linkId': '뉴발란스토앤토'},
   {'rank': 4, 'keyword': '양산', 'linkId': '양산'},
   {'rank': 5, 'keyword': '슬리퍼', 'linkId': '슬리퍼'},
   {'rank': 6, 'keyword': '크록스샌들', 'linkId': '크록스샌들'},
   {'rank': 7, 'keyword': '아쿠아슈즈', 'linkId': '아쿠아슈즈'},
   {'rank': 8, 'keyword': '샌들', 'linkId': '샌들'},
   {'rank': 9, 'keyword': '모자', 'linkId': '모자'},
   {'rank': 10, 'keyword': '우포스쪼리', 'linkId': '우포스쪼리'}]},
 {'message': None,
  'statusCode': 200,
  'returnCode': 0,
  'date': '2021/06/29',
  'datetime': '2021.06.29.(화)',
  'range': '',
  'ranks': [{'rank': 1, 'keyword': '크록스', 'linkId': '크록스'},
   {'rank': 2, 'keyword': '나이키운동화', 'linkId': '나이키운동화'},
   {'rank': 3, 'keyword': '뉴발란스토앤토', 'linkId': '뉴발란스토앤토'},
   {'rank': 4, 'keywor

In [82]:
data[0]['ranks'][0]['keyword']

'크록스'

In [83]:
# 위의 json을 배열형태로 바꾼 이러한 파일들을 pandas의 dataframe으로 변환 가능
# json이 판다스로 바꾸기 쉬워서 좋음

import pandas as pd
from pandas.io.json import json_normalize

df = pd.DataFrame(data[0])
df

,message,statusCode,returnCode,date,datetime,range,ranks
0,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 1, 'keyword': '크록스', 'linkId': '크록스'}"
1,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 2, 'keyword': '나이키운동화', 'linkId': '나이..."
2,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 3, 'keyword': '뉴발란스토앤토', 'linkId': '뉴..."
3,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 4, 'keyword': '양산', 'linkId': '양산'}"
4,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 5, 'keyword': '슬리퍼', 'linkId': '슬리퍼'}"
5,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 6, 'keyword': '크록스샌들', 'linkId': '크록스..."
6,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 7, 'keyword': '아쿠아슈즈', 'linkId': '아쿠아..."
7,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 8, 'keyword': '샌들', 'linkId': '샌들'}"
8,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 9, 'keyword': '모자', 'linkId': '모자'}"
9,None,200,0,2021/06/28,2021.06.28.(월),,"{'rank': 10, 'keyword': '우포스쪼리', 'linkId': '우포..."
